# Tokenization and Embeddings

In [23]:
import openai
import os

from dotenv import load_dotenv
load_dotenv()

from unstructured.partition.text import partition_text
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

In [32]:
from unstructured.cleaners.core import group_broken_paragraphs
elements = partition_text('data/Stein.txt', paragraph_grouper=group_broken_paragraphs)

In [33]:
print("\n\n".join([str(el) for el in elements][:10]))


Mr und Mrs Dursley im Ligusterweg Nummer 4 waren stolz darauf, ganz und gar normal zu sein, sehr stolz sogar. Niemand wäre auf die Idee gekommen, sie könnten sich in eine merkwürdige und geheimnisvolle Geschichte verstricken, denn mit solchem Unsinn wollten sie nichts zu tun haben. Mr Dursley war Direktor einer Firma namens Grunnings, die Bohrmaschinen herstellte. Er war groß und bullig und hatte fast keinen Hals, dafür aber einen sehr großen Schnurrbart. Mrs Dursley war dünn und blond und besaß doppelt so viel Hals, wie notwendig gewesen wäre, was allerdings sehr nützlich war, denn so konnte sie den Hals über den Gartenzaun recken und zu den Nachbarn hinüberspähen. Die Dursleys hatten einen kleinen Sohn namens Dudley und in ihren Augen gab es nirgendwo einen prächtigeren Jungen.

Die Dursleys besaßen alles, was sie wollten, doch sie hatten auch ein Geheimnis, und dass es jemand aufdecken könnte, war ihre größte Sorge. Einfach unerträglich wäre es, wenn die Sache mit den Potters heraus

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
story_texts = [str(el) for el in elements]
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(story_texts))
print(f"\nTotal chunks: {len(character_split_texts)}")


Total chunks: 744


In [55]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=128, model_name="paraphrase-multilingual-MiniLM-L12-v2")

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(f"\nTotal chunks: {len(token_split_texts)}")
print(len(token_split_texts[0]))


Total chunks: 2
27


In [39]:
print(token_split_texts[0])

Mr und Mrs Dursley im Ligusterweg Nummer 4 waren stolz darauf, ganz und gar normal zu sein, sehr stolz sogar. Niemand wäre auf die Idee gekommen, sie könnten sich in eine merkwürdige und geheimnisvolle Geschichte verstricken, denn mit solchem Unsinn wollten sie nichts zu tun haben. Mr Dursley war Direktor einer Firma namens Grunnings, die Bohrmaschinen herstellte. Er war groß und bullig und hatte fast keinen Hals, dafür aber einen sehr großen Schnurrbart. Mrs Dursley war dünn und blond und besaß doppelt so viel Hals


In [42]:
from sentence_transformers import SentenceTransformer, models
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)
tokenized_chunks = []
for text in character_split_texts[:2]:
    # Tokenize each chunk
    encoded_input = model.tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='pt')
    # Convert token IDs back to tokens
    tokens = model.tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0].tolist())
    tokenized_chunks.append(tokens)

# Print the tokenized chunks
for i, tokens in enumerate(tokenized_chunks):
    print(f"Chunk {i}: {tokens}")



Chunk 0: ['<s>', '▁Mr', '▁und', '▁Mrs', '▁Dur', 's', 'ley', '▁im', '▁Lig', 'u', 'ster', 'weg', '▁Nummer', '▁4', '▁waren', '▁stolz', '▁darauf', ',', '▁ganz', '▁und', '▁gar', '▁normal', '▁zu', '▁sein', ',', '▁sehr', '▁stolz', '▁sogar', '.', '▁Niemand', '▁wäre', '▁auf', '▁die', '▁Idee', '▁gekommen', ',', '▁sie', '▁könnten', '▁sich', '▁in', '▁eine', '▁merk', 'würdig', 'e', '▁und', '▁geheim', 'nis', 'volle', '▁Geschichte', '▁vers', 'trick', 'en', ',', '▁denn', '▁mit', '▁solche', 'm', '▁Un', 'sinn', '▁wollte', 'n', '▁sie', '▁nichts', '▁zu', '▁tun', '▁haben', '.', '▁Mr', '▁Dur', 's', 'ley', '▁war', '▁Direktor', '▁einer', '▁Firma', '▁namen', 's', '▁Grunn', 'ings', ',', '▁die', '▁Boh', 'r', 'maschine', 'n', '▁herstel', 'lte', '.', '▁Er', '▁war', '▁groß', '▁und', '▁bul', 'lig', '▁und', '▁hatte', '▁fast', '▁keinen', '▁Hal', 's', ',', '▁dafür', '▁aber', '▁einen', '▁sehr', '▁großen', '▁Sch', 'nur', 'r', 'bart', '.', '▁Mrs', '▁Dur', 's', 'ley', '▁war', '▁dün', 'n', '▁und', '▁blond', '▁und', '▁be', '

In [51]:
from sentence_transformers import SentenceTransformer, models
model_name = "Sahajtomar/German-semantic"
model = SentenceTransformer(model_name)
tokenized_chunks = []
for text in character_split_texts[:2]:
    # Tokenize each chunk
    encoded_input = model.tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='pt')
    # Convert token IDs back to tokens
    tokens = model.tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0].tolist())
    tokenized_chunks.append(tokens)

# Print the tokenized chunks
for i, tokens in enumerate(tokenized_chunks):
    print(f"Chunk {i}: {tokens}")
    print(len(tokens))

Chunk 0: ['[CLS]', 'Your', 'first', 'te', '##xt', 'ch', '##unk', 'here', '.', '[SEP]']
10
Chunk 1: ['[CLS]', 'Your', 'sec', '##ond', 'te', '##xt', 'ch', '##unk', 'here', '.', '[SEP]']
11


In [52]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction(model_name="paraphrase-multilingual-MiniLM-L12-v2")
print(embedding_function([token_split_texts[10]]))
print(len(embedding_function([token_split_texts[10]])[0]))
print(token_split_texts[10])

[[0.23705792427062988, 0.17953404784202576, 0.008761091157793999, 0.06866278499364853, 0.18859246373176575, 0.025030042976140976, 0.20453552901744843, 0.0856434628367424, -0.12013310939073563, -0.1756037026643753, 0.21593599021434784, 0.17754752933979034, -0.06816255301237106, 0.15382297337055206, 0.03742352873086929, 0.02565864473581314, -0.03674404323101044, 0.09956962615251541, -0.11178182810544968, 0.01585395261645317, -0.14351488649845123, 0.03620703145861626, 0.03634040802717209, 0.13937081396579742, -0.014753739349544048, -0.17134839296340942, 0.10067982226610184, -0.0721849724650383, -0.09251675009727478, -0.20344743132591248, -0.0860968604683876, -0.11146057397127151, 0.20581139624118805, 0.06186888739466667, -0.06888921558856964, -0.021220432594418526, 0.2979533076286316, 0.14489932358264923, 0.14279508590698242, 0.23565231263637543, -0.1417035609483719, -0.10818491131067276, -0.12070940434932709, -0.00885545089840889, -0.026414407417178154, -0.1322002410888672, -0.0280905831

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("attelsander_sample.pdf", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()